### dbSNFP release notes checker
Here we will create an Agent capable of fetching latest release notes from Gnomad and provide summary

In [ ]:
# Import libraries
from dotenv import load_dotenv
from agents import Runner, trace, function_tool, Agent, OpenAIChatCompletionsModel, SQLiteSession
from openai import AsyncOpenAI
import os, requests, time, re
from bs4 import BeautifulSoup
from datetime import datetime


#### Load openai api key from environment

In [ ]:
# Load environment variables
load_dotenv(override=True)

#### Define LLM model

In [ ]:
# Define LLM model

# Define the OpenAI client pointing to a local LLM server
client = AsyncOpenAI(base_url="http://localhost:11434/v1")

# Define model using the local LLM server client
# use gpt-oss which is better for tool usage and reasoning even though it takes time
model = OpenAIChatCompletionsModel(model = "gpt-oss",openai_client= client)


#### Define Tool

In [ ]:

@function_tool
def fetch_dbnsfp_releases():
    """ Fetch the DBNSFP releases page content. """
    url = "https://www.dbnsfp.org/releases"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    # extract all texts from html page
    texts = soup.get_text()
    return texts


#### Define Agent

In [ ]:
instructions_dbnsfp = (
        "You are a specialized agent for dbNSFP. Your task is to read the latest release notes from https://www.dbnsfp.org/releases and produce a concise, well-formatted markdown summary containing the latest release version, release date if any, and a bulleted list of major changes or updates"
    )

dbnsfp_agent = Agent(
        name="dbnsfp_agent",
        instructions=instructions_dbnsfp,
        model=model,
        tools=[fetch_dbnsfp_releases]
    )


#### Run Agent in Chat mode with session

In [ ]:
# Define a session to maintain context across interactions
session = SQLiteSession("edsp_agent_session.db")

# Can leverage chat interface for multiple turns
async def chat(message, history):
    # pass session object to maintain context
    result = await Runner.run(dbnsfp_agent, message, session=session)
    return result.final_output

import gradio as gr
gr.ChatInterface(
    chat,
    title="dbNSFP Release Notes Checker",
    description="Chat with the agent to get release notes summaries from dbNSFP database."
).launch()